## Token Tracker
made by eyeol

띄어쓰기 전처리 유무에 따른 tokenizing 결과를 비교해보고 싶어서 만들었음

**주의점**

파일 실행은 root 폴더(train.py와 동일한 위치)에서 해야 합니다<br/>
baseline_config.yaml에서 inference의 model path를 원하는 모델 경로로 바꿔야 합니다


### 1. 모델 및 토크나이저 준비

In [12]:
import argparse
import yaml
import pandas as pd
import os
from tqdm.auto import tqdm

import torch
import transformers
import pandas as pd

import pytorch_lightning as pl
#import wandb
##############################
from utils import data_pipeline
from model.model import Model

In [9]:
parser = argparse.ArgumentParser()
parser.add_argument('--train_path', default='./data/raw/train.csv')
parser.add_argument('--dev_path', default='./data/raw/dev.csv')
parser.add_argument('--test_path', default='./data/raw/dev.csv')
##
parser.add_argument('--predict_path', default='./data/raw/dev.csv')
## predict_path에 dev.csv로 설정 >> dev set에 대한 predictions 출력
args = parser.parse_args(args=[])

# baseline_config 설정 불러오기
with open('baselines/baseline_config.yaml', 'r', encoding='utf-8') as f:
        CFG = yaml.load(f, Loader=yaml.FullLoader)

# inference에 쓸 모델 불러오기(CFG로 참조)
model_path = CFG['inference']['model_path']

In [10]:
dataloader = data_pipeline.Dataloader(CFG, args.train_path, args.dev_path, args.test_path, args.predict_path)
model = torch.load(model_path)

In [11]:
model

Model(
  (plm): ElectraForSequenceClassification(
    (electra): ElectraModel(
      (embeddings): ElectraEmbeddings(
        (word_embeddings): Embedding(30000, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): ElectraEncoder(
        (layer): ModuleList(
          (0-11): 12 x ElectraLayer(
            (attention): ElectraAttention(
              (self): ElectraSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): ElectraSelfOutput(
                (dense): Linear(in_features=768, o

In [13]:
## Tokenizer로 쓸 ElectraTokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained('snunlp/KR-ELECTRA-discriminator', max_length=160)

In [14]:
tokenizer

ElectraTokenizerFast(name_or_path='snunlp/KR-ELECTRA-discriminator', vocab_size=30000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [15]:
text_columns = ['sentence_1', 'sentence_2']

In [16]:
def tokenizing(dataframe):
    data = []
    for idx, item in tqdm(dataframe.iterrows(), desc='tokenizing', total=len(dataframe)):
        # 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
        text = '[SEP]'.join([item[text_column] for text_column in text_columns])
        # padding=True와 truncation=True 옵션을 명시적으로 추가합니다.
        outputs = tokenizer(
            text,
            add_special_tokens=True,
            padding='max_length',  # max_length로 패딩을 고정
            truncation=True,       # 텍스트를 최대 길이로 자름
            max_length=160         # max_length 설정
        )

        token_ids = outputs['input_ids']
        tokens = tokenizer.convert_ids_to_tokens(token_ids)


        data.append(tokens)
    return data

### 2. 데이터셋 준비

In [39]:
df = pd.read_csv('./data/raw/train.csv')

## spacing 유무에 따른 token 비교를 위해 원본 남겨두기
df_origin = pd.read_csv('./data/raw/train.csv')

In [40]:
# spacing에 쓸 kiwi
from kiwipiepy import Kiwi

kiwi = Kiwi()

def correct_spacing(text):
    return kiwi.space(text)

In [41]:
# DataFrame에 apply를 사용해 띄어쓰기 교정 적용
df['sentence_1'] = df['sentence_1'].apply(correct_spacing)
df['sentence_2'] = df['sentence_2'].apply(correct_spacing)

In [43]:
df.sample(n=5, random_state=11)

,id,source,sentence_1,sentence_2,label,binary-label
5123,boostcamp-sts-v1-train-5123,petition-rtt,오프라인 오픈형 성인용품점을 업종 관리해 주세요.,오프라인 오픈형 성인 용품 매장을 관리해 주세요.,4.2,1.0
1768,boostcamp-sts-v1-train-1768,petition-rtt,"유치원, 어린이집 영어 수업 금지법 철회해 주시고 유치원, 어린이집 영어 강사들을 ...",유치원과 어린이집에서 영어 수업을 금지하는 법을 철회하고 유치원과 어린이집에서 영어...,4.4,1.0
5608,boostcamp-sts-v1-train-5608,nsmc-sampled,반전 영화인 걸 알면서 봤는데 이렇게 뒷 통수를 칠 줄이야 ㅎㅎ,중학교 때 반전이란 걸 알게해 준 영화,0.8,0.0
353,boostcamp-sts-v1-train-353,slack-sampled,부끄럽다고 사진에는 안 나왔습니다.,메일로는 안 왔어요 ~,0.0,0.0
7278,boostcamp-sts-v1-train-7278,nsmc-sampled,평점이 너무 높다 이 정돈 아님,평점이 너무 낮은 듯;;;,0.8,0.0


된거 같은데?

In [42]:
df_origin.sample(n=5, random_state=11)

,id,source,sentence_1,sentence_2,label,binary-label
5123,boostcamp-sts-v1-train-5123,petition-rtt,오프라인 오픈형 성인용품점을 업종관리 해주세요.,오프라인 오픈형 성인용품 매장을 관리해주세요.,4.2,1.0
1768,boostcamp-sts-v1-train-1768,petition-rtt,"유치원, 어린이집 영어수업 금지법 철회 해주시고 유치원, 어린이집 영어강사들을 보호...",유치원과 어린이집에서 영어 수업을 금지하는 법을 철회하고 유치원과 어린이집에서 영어...,4.4,1.0
5608,boostcamp-sts-v1-train-5608,nsmc-sampled,반전영화인걸 알면서 봤는데 이렇게 뒷통수를 칠줄이야 ㅎㅎ,중학교때 반전이란걸 알게해준 영화,0.8,0.0
353,boostcamp-sts-v1-train-353,slack-sampled,부끄럽다고 사진에는 안나왔습니다.,메일로는 안왔어요 ~,0.0,0.0
7278,boostcamp-sts-v1-train-7278,nsmc-sampled,평점이 너무 높다 이정돈아님,평점이 너무 낮은 듯;;;,0.8,0.0


원본과 비교하면 spacing이 적절하게 처리된것 같다

In [45]:
tokenized_spaced = tokenizing(dataframe=df)
tokenized_origin = tokenizing(dataframe=df_origin)

tokenizing: 100%|██████████| 9324/9324 [00:01<00:00, 7902.71it/s]


In [46]:
print(tokenized_origin[1])
print(tokenized_spaced[1])

['[CLS]', '앗', '제', '##가', '접근', '##권', '##한', '##이', '없', '##다고', '뜹', '##니다', ';', ';', '[SEP]', '오', ',', '액', '##세스', '권한', '##이', '없', '##다고', '합니다', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

spacing에 따라 토큰이 달라지는 것을 확인

이 차이가 임베딩 이후에 어떤 결과 차이로 이어지는지까지 확인하고 싶었는데 실력 이슈로 보류 </br>
다음 프로젝트에서는 임베딩 벡터의 변화 관찰, attention map 출력까지 해보고 싶다